## Ejemplo: Datos de los Estados de EE.UU.

Las operaciones merge y join aparecen con más frecuencia cuando se combinan datos de distintas fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE.UU. y su población.
Los archivos de datos se encuentran en http://github.com/jakevdp/data-USstates/:

In [2]:
# Los siguientes son comandos shell para descargar los datos
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

In [3]:
import pandas as pd
import numpy as np

Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

In [29]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

In [30]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [31]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [32]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


Con esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE.UU. según su densidad de población en 2010.
Es evidente que disponemos de los datos necesarios para obtener este resultado, pero tendremos que combinar los conjuntos de datos para obtenerlo.

Empezaremos con una combinación múltiple que nos dará el nombre completo del estado dentro del ``DataFrame`` de población.

Queremos hacer la combinación basándonos en la columna ``state/region`` de ``pop`` y en la columna ``abbreviation`` de ``abbrevs``.

Utilizaremos ``how='outer'`` para asegurarnos de que no se pierde ningún dato por no coincidir las etiquetas.

In [5]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation', indicator=True)
merged = merged.drop('abbreviation', axis=1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state,_merge
0,AL,under18,2012,1117489.0,Alabama,both
1,AL,total,2012,4817528.0,Alabama,both
2,AL,under18,2010,1130966.0,Alabama,both
3,AL,total,2010,4785570.0,Alabama,both
4,AL,under18,2011,1125763.0,Alabama,both


Comprobemos si se ha producido alguna discordancia, lo que podemos hacer buscando filas con nulos para ello usamos el método any().

any() vs all()

In [39]:
### any() y all()
ejemplo = pd.Series([1, 5, 6, np.nan, 4, 5])


True

In [48]:
# ¿Alguno es no nulo?
ejemplo.notnull().any()

True

In [50]:
# ¿Alguno es nulo?
ejemplo.isnull().any()

True

In [49]:
# Todos son no nulos?
ejemplo.notnull().all()

False

In [51]:
# Todos son nulos?
ejemplo.isnull().all()

False

In [44]:
### any() y all()
ejemplo2 = pd.Series([1, 5, 6, 5, 4, 5])

In [45]:
# Alguno es no nulo?
ejemplo2.notnull().any()

True

In [52]:
# Alguno es  nulo?
ejemplo2.isnull().any()

False

In [47]:
# Todos son no nulo?
ejemplo2.notnull().all()

True

In [53]:
# Todos son nulos?
ejemplo2.isnull().all()

False

In [56]:
# ejemplo3 podría ser una columna vacia
ejemplo3 = pd.Series([np.nan, np.nan])


In [57]:
# ¿Alguno es no nulo?
ejemplo3.notnull().any()

False

In [58]:
# Todos son no nulo?
ejemplo3.notnull().all()

False

In [59]:
# ¿Alguno es nulo?
ejemplo3.isnull().any()

True

In [60]:
# ¿Todos son nulos?
ejemplo3.isnull().all()

True

In [61]:
### any() y all()
ejemplo4 = pd.Series([1, 5, 6, 8, 4, 5])

In [65]:
(ejemplo4>4).any()

True

In [66]:
(ejemplo4>4).all()

False

In [67]:
(ejemplo4<10).any()

True

In [68]:
(ejemplo4<10).all()

True

In [6]:
# any() busca algún True en una columna
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
_merge          False
dtype: bool

In [33]:
# all() busca algún True en una columna
merged.isnull().all()

state/region    False
ages            False
year            False
population      False
state           False
outer           False
dtype: bool

Algunos de los datos de ``población`` son nulos; ¡averigüemos cuáles son!

In [7]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state,_merge
2448,PR,under18,1990,NaN,NaN,left_only
2449,PR,total,1990,NaN,NaN,left_only
2450,PR,total,1991,NaN,NaN,left_only
2451,PR,under18,1991,NaN,NaN,left_only
2452,PR,total,1993,NaN,NaN,left_only


In [8]:
merged[merged['_merge']!="both"]

,state/region,ages,year,population,state,_merge
2448,PR,under18,1990,NaN,NaN,left_only
2449,PR,total,1990,NaN,NaN,left_only
2450,PR,total,1991,NaN,NaN,left_only
2451,PR,under18,1991,NaN,NaN,left_only
2452,PR,total,1993,NaN,NaN,left_only
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,left_only
2540,USA,under18,2011,73902222.0,NaN,left_only
2541,USA,total,2011,311582564.0,NaN,left_only
2542,USA,under18,2012,73708179.0,NaN,left_only


In [9]:
merged.groupby("_merge").count()

C:\Users\jonat\AppData\Local\Temp\ipykernel_4244\1046811352.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.groupby("_merge").count()


,state/region,ages,year,population,state
_merge,,,,,
left_only,96,96,96,76,0
right_only,0,0,0,0,0
both,2448,2448,2448,2448,2448


Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; esto se debe probablemente a que estos datos no están disponibles en la fuente original.

Y lo que es más importante, vemos también que algunas de las nuevas entradas ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [10]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Podemos deducir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (USA), mientras que estas entradas no aparecen en la clave de abreviatura del estado.
Podemos solucionarlo rápidamente introduciendo las entradas adecuadas:

In [11]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
_merge          False
dtype: bool

Se acabaron los nulos en la columna ``state``: ¡listo!

Ahora podemos fusionar el resultado con los datos del área mediante un procedimiento similar.
Al examinar nuestros resultados, querremos unirlos en la columna ``state`` de ambos:

In [12]:
merged.rename(columns={"_merge":"outer"}, inplace=True)
final = pd.merge(merged, areas, on='state', how='left', indicator=True)
final.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
0,AL,under18,2012,1117489.0,Alabama,both,52423.0,both
1,AL,total,2012,4817528.0,Alabama,both,52423.0,both
2,AL,under18,2010,1130966.0,Alabama,both,52423.0,both
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
4,AL,under18,2011,1125763.0,Alabama,both,52423.0,both


Breves notas sobre la igualdad entre dataframes

In [13]:
# NaN != NaN
(final != final).sum()

state/region      0
ages              0
year              0
population       20
state             0
outer             0
area (sq. mi)    48
_merge            0
dtype: int64

In [14]:
# Usamos el metodo equals.
# Más adelante veremos un ejemplo comparando dos dataframes distintos!
final.equals(final)

True

In [15]:
final.groupby('_merge')['_merge'].count()

C:\Users\jonat\AppData\Local\Temp\ipykernel_4244\3929008639.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final.groupby('_merge')['_merge'].count()


_merge
left_only       48
right_only       0
both          2496
Name: _merge, dtype: int64

De nuevo, vamos a comprobar si hay nulos para ver si hay algún desajuste:

In [16]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
outer            False
area (sq. mi)     True
_merge           False
dtype: bool

Hay nulos en la columna ``área``; podemos echar un vistazo para ver qué regiones se ignoraron aquí:

In [17]:
final[final["_merge"]!="both"]

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
2496,USA,under18,1990,64218512.0,United States,left_only,NaN,left_only
2497,USA,total,1990,249622814.0,United States,left_only,NaN,left_only
2498,USA,total,1991,252980942.0,United States,left_only,NaN,left_only
2499,USA,under18,1991,65313018.0,United States,left_only,NaN,left_only
2500,USA,under18,1992,66509177.0,United States,left_only,NaN,left_only
2501,USA,total,1992,256514231.0,United States,left_only,NaN,left_only
2502,USA,total,1993,259918595.0,United States,left_only,NaN,left_only
2503,USA,under18,1993,67594938.0,United States,left_only,NaN,left_only
2504,USA,under18,1994,68640936.0,United States,left_only,NaN,left_only
2505,USA,total,1994,263125826.0,United States,left_only,NaN,left_only


In [18]:
final[final["_merge"]!="both"]['state/region'].unique()

array(['USA'], dtype=object)

Vemos que nuestro ``DataFrame`` de ``areas`` no contiene el área de Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (usando la suma de las áreas de todos los estados, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [19]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
0,AL,under18,2012,1117489.0,Alabama,both,52423.0,both
1,AL,total,2012,4817528.0,Alabama,both,52423.0,both
2,AL,under18,2010,1130966.0,Alabama,both,52423.0,both
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
4,AL,under18,2011,1125763.0,Alabama,both,52423.0,both


Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta que nos interesa, vamos a seleccionar primero la parte de los datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()`` para hacerlo rápidamente (esto requiere tener instalado el paquete ``numexpr``:

In [20]:
data2010 = final[(final["year"] == 2010) & (final["ages"] == 'total')]
data2010.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
91,AK,total,2010,713868.0,Alaska,both,656425.0,both
101,AZ,total,2010,6408790.0,Arizona,both,114006.0,both
189,AR,total,2010,2922280.0,Arkansas,both,53182.0,both
197,CA,total,2010,37333601.0,California,both,163707.0,both


In [21]:
data2010_q = final.query("year == 2010 & ages == 'total'")
data2010_q.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both
91,AK,total,2010,713868.0,Alaska,both,656425.0,both
101,AZ,total,2010,6408790.0,Arizona,both,114006.0,both
189,AR,total,2010,2922280.0,Arkansas,both,53182.0,both
197,CA,total,2010,37333601.0,California,both,163707.0,both


In [98]:
# limitaciones: nombres de las columnas con símbolos
#final.query("year == 2010 & state/region == 'AL'")
final.rename(columns={"state/region":"region"}).query("year == 2010 & region == 'AL'")

,region,ages,year,population,state,outer,area (sq. mi),_merge
2,AL,under18,2010,1130966.0,Alabama,both,52423.0,both
3,AL,total,2010,4785570.0,Alabama,both,52423.0,both


In [22]:
data2010.equals(data2010_q)

True

In [23]:
(data2010 != data2010_q).sum()

state/region     0
ages             0
year             0
population       0
state            0
outer            0
area (sq. mi)    0
_merge           0
dtype: int64

In [70]:
# El orden importa en las comparaciones entre dataframes!!
data2010 = data2010.sample(frac=1,replace=False)
#(data2010 != data2010_q).sum()

In [71]:
data2010.equals(data2010_q)

False

Ahora vamos a calcular la densidad de población y mostrarla por orden.
Empezaremos por volver a indexar nuestros datos en el estado y, a continuación, calcularemos el resultado:

In [26]:
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

389     8898.897059
2490    1058.665149
1445    1009.253268
1914     681.339159
293      645.600649
dtype: float64

In [92]:
data2010['density'] = data2010['population'] / data2010['area (sq. mi)']
data2010.sort_values(by=["density"],ascending=False, inplace=True)
data2010.head()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge,density
389,DC,total,2010,605125.0,District of Columbia,both,68.0,both,8898.897059
2490,PR,total,2010,3721208.0,Puerto Rico,left_only,3515.0,both,1058.665149
1445,NJ,total,2010,8802707.0,New Jersey,both,8722.0,both,1009.253268
1914,RI,total,2010,1052669.0,Rhode Island,both,1545.0,both,681.339159
293,CT,total,2010,3579210.0,Connecticut,both,5544.0,both,645.600649


El resultado es una clasificación de los estados de EE.UU. más Washington, DC, y Puerto Rico por orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington, DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos comprobar el final de la lista:

In [93]:
data2010.tail()

,state/region,ages,year,population,state,outer,area (sq. mi),_merge,density
2010,SD,total,2010,816211.0,South Dakota,both,77121.0,both,10.583512
1637,ND,total,2010,674344.0,North Dakota,both,70704.0,both,9.537565
1253,MT,total,2010,990527.0,Montana,both,147046.0,both,6.736171
2405,WY,total,2010,564222.0,Wyoming,both,97818.0,both,5.768079
91,AK,total,2010,713868.0,Alaska,both,656425.0,both,1.087509


In [82]:
density.tail()

2010    10.583512
1637     9.537565
1253     6.736171
2405     5.768079
91       1.087509
dtype: float64

Vemos que el estado menos denso, con diferencia, es Alaska, con una media de poco más de un residente por milla cuadrada.

Este tipo de fusión de datos desordenados es una tarea habitual cuando se intenta responder a preguntas utilizando fuentes de datos del mundo real.
Espero que este ejemplo te haya dado una idea de las formas en que puedes combinar las herramientas que hemos visto para obtener información de tus datos.